In [117]:
import cv2
import mediapipe as mp
import numpy as np
import phoenix as px
from opentelemetry import trace
from phoenix.otel import register
from opentelemetry.trace import Status, StatusCode

In [118]:
px.launch_app(port=6007)
tracer_provider = register(
    project_name="pushup-app",
    endpoint="http://localhost:6007/v1/traces",
    batch=False
)
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


🌍 To view the Phoenix app in your browser, visit http://localhost:6007/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: pushup-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6007/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [119]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [120]:
class BaseModel:
    def load(self):
        raise NotImplementedError

    def predict(self, frame):
        raise NotImplementedError

    def draw(self, frame, results):
        return frame

In [121]:
class MediaPipePushupModel(BaseModel):
    def __init__(self):
        self.pose = None
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose

    def load(self):
        self.pose = self.mp_pose.Pose()

    def predict(self, frame):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return self.pose.process(rgb)

    def draw(self, frame, results):
        if results.pose_landmarks:
            self.mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS
            )
        return frame

In [122]:
def process_frame(frame, model, counter, stage, frame_idx):
    with tracer.start_as_current_span("frame_processing") as span:
        try:
            # Initialize pipeline attributes
            pipeline_attrs = {
                "landmarks_detected": False,
                "shoulder": None,
                "elbow": None,
                "wrist": None,
                "angle_arm": None,
                "angle_hip": None,
                "counter": counter,
                "stage": stage,
                "incorrect_pose": None,
            }

            span.set_attribute("frame.index", frame_idx)
            span.set_attribute("frame.shape", str(frame.shape))

            # Inference
            results = model.predict(frame)
            annotated = frame.copy()
            annotated = model.draw(annotated, results)

            if results.pose_landmarks:
                pipeline_attrs["landmarks_detected"] = True
                landmarks = results.pose_landmarks.landmark

                shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                         landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x,
                         landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
                hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]
                knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
                ankle = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].y]


                pipeline_attrs["shoulder"] = shoulder
                pipeline_attrs["elbow"] = elbow
                pipeline_attrs["wrist"] = wrist

                # Calculate angles
                angle_arm = calculate_angle(shoulder, elbow, wrist)
                angle_hip = calculate_angle(hip, knee, ankle)

                pipeline_attrs["angle_arm"] = angle_arm
                pipeline_attrs["angle_hip"] = angle_hip

                # Push-up counting logic
                if stage is None:
                    stage = "up"  # Initialize stage once if None

                if angle_arm > 160:
                    stage = "up"
                if angle_arm < 70 and stage == "up":
                    stage = "down"
                    counter += 1
                    print(f"Push-up Count: {counter}")

                pipeline_attrs["counter"] = counter
                pipeline_attrs["stage"] = stage

                # Incorrect pose check
                if abs(angle_hip) < 170 :
                    pipeline_attrs['incorrect_pose'] = True
                else:
                    pipeline_attrs['incorrect_pose'] = False

                # Draw overlay rectangle and text on annotated frame (displayed frame)
            cv2.rectangle(annotated, (0,0), (225,130), (245,117,16), -1)
            cv2.putText(annotated, 'Push-ups', (15, 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)
            cv2.putText(annotated, str(counter), (10, 65),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)
            cv2.putText(annotated, f'Arm Angle: {angle_arm:.2f}' if angle_arm is not None else 'Arm Angle: --',
                        (10, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            cv2.putText(annotated, f'Hip Angle: {angle_hip:.2f}' if angle_hip is not None else 'Hip Angle: --',
                        (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            cv2.putText(annotated, f'Stage: {stage}', (10, 145),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            if pipeline_attrs["incorrect_pose"]:
                cv2.putText(annotated, 'Incorrect Pose -Keep your body straight', (10, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

            # Log all pipeline attributes into Phoenix
            for k, v in pipeline_attrs.items():
                span.set_attribute(f"pipeline.{k}", str(v))

            span.set_status(Status(StatusCode.OK))
            return annotated, counter, stage, pipeline_attrs

        except Exception as e:
            span.record_exception(e)
            span.set_status(Status(StatusCode.ERROR, str(e)))
            return frame, counter, stage, pipeline_attrs



In [123]:
def run_experiment(model: BaseModel, video_source):
    cap = cv2.VideoCapture(video_source)
    model.load()

    counter = 0
    stage = None
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        annotated, counter, stage, pipeline_attrs = process_frame(frame, model, counter, stage, frame_idx)
        frame_idx += 1

        cv2.imshow("Pushup Experiment", annotated)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [124]:
def get_model(name):
    if name == "mediapipe":
        return MediaPipePushupModel()
    else:
        raise ValueError(f"Unknown model: {name}")

In [125]:
if __name__ == "__main__":
    model = get_model("mediapipe")
    run_experiment(model ,"/Users/priyanka./Documents/agentic-security/Mediapipe/push-up_1.mp4" )

I0000 00:00:1758699637.184335 10425696 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
W0000 00:00:1758699637.307175 10446696 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758699637.329528 10446703 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Push-up Count: 1
Push-up Count: 2
Push-up Count: 3
